<a href="https://colab.research.google.com/github/stoianmihail/HyperTone/blob/main/HyperTone2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# lstm model
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [2]:
hypertone_dir = '/content/drive/MyDrive/hypertone/'
preprocessing_dir = hypertone_dir + 'preprocessing/'

In [3]:
import os
def get_files(path):
  ret = []
  for root, _, files in os.walk(path, topdown=False):
    for file in files:
      ret.append({'path': os.path.join(root,file), 'data' : None})
  return ret

In [4]:
import os
import pickle
import pandas as pd

def compress(a):
  return a[np.logical_or(np.insert(np.diff(a).astype(bool), 0, True), np.asarray(a, dtype=bool))]

def parse_songs():
  songs = {}
  for file in get_files(preprocessing_dir):
    if '.pkl' not in file['path']:
      continue 
    song_name = os.path.basename(file['path']).replace('.pkl', '')
    x = pd.read_pickle(file['path'])
    compressed = compress(np.asarray(x[1]))
    
    # TODO: this is only for these recordings
    # TODO: Next time, also save the label!
    tone = 5
    if song_name[0].isdigit():
      tone = int(song_name[0])
    if tone == 1 or tone == 8:# or tone == 5 or tone == 3:
      songs[song_name] = {'x': compressed, 'y': tone}
  return songs

songs = parse_songs()
print(songs['8.2 Troparele'])

{'x': array([-1,  1,  0, -1,  0,  1, -1,  0,  2, -1, -1, -1,  0,  4, -5,  1, -2,
        0,  1,  0, -1,  0, -1,  0,  4,  1,  1,  1,  0, -1,  0, -1,  0, -1,
        1,  0, -6,  2,  1,  1,  1,  0,  1,  0, -1,  0, -1,  0,  1,  0, -1,
        0, -7,  4,  1,  1,  1,  0,  1, -1,  0, -1, -1, -1,  0,  1,  1,  0,
       -1, -1,  0,  1,  1,  1,  0,  1, -1, -1, -1,  0, -1,  0,  1,  0,  2,
        1,  1,  0, -1,  0,  1,  0, -1,  0,  1,  0, -1,  0, -1,  0,  1,  0,
        1,  0, -1,  0,  1,  1,  0, -1,  0, -1,  0,  1,  1,  0, -1,  0, -1,
        0,  1,  0,  1,  2, -6,  1,  3,  0,  2,  1,  1,  1,  1,  0,  1,  0,
        1,  0,  1,  0, -1,  0,  1,  0, -1,  0,  1,  0, -1,  1,  0,  1,  0,
       -9,  2,  0,  1,  0,  1,  0, -1,  0,  1,  0,  5, -2,  0,  1, -1,  0,
       -1,  3, -2,  8, -4,  0,  1,  1,  1,  0, -1, -1,  0, -1,  0,  1,  0,
       -1,  0, -2,  2, -5, -1,  2,  2,  1,  0, -1, -2,  0,  1,  0,  1, -1,
        0, -2, -5,  0,  1,  1,  0,  2,  1,  0,  2,  0, -1,  1,  0, -1, -1,
        0, -2,  0, 

In [5]:
import tensorflow.keras as keras
kSequenceLength = 128

def update(y):
  if y == 1:
    return 0
  return 1

def generate_training_sequences(seq_len=kSequenceLength):
  # load songs and map them to int
  songs = parse_songs()

  inputs, targets = [], []
  for song in songs:
    x = songs[song]['x']
    y = songs[song]['y']
    y = update(y)
    #x[np.insert(np.diff(x).astype(np.bool), 0, True)]
    if len(x) < seq_len:
      x.resize(seq_len)
    num_sequences = len(x) - seq_len + 1
    for index in range(num_sequences):
      inputs.append(np.asarray(x[index : index + seq_len]).reshape((seq_len, 1)))
      targets.append(y)

  # one-hot encode the sequences  
  # inputs size: (# of sequences, sequence length, max(diff))
  #inputs = keras.utils.to_categorical(inputs, num_classes=kOutputUnits)
  inputs = np.asarray(inputs)
  print(inputs.shape)
  #n = len(inputs)
  #inputs = np.reshape(inputs, (n, seq_len, 1))
  # normalize input
  #inputs = inputs / float(kOutputUnits)
  
  targets = keras.utils.to_categorical(targets, num_classes=2)
  print(targets.shape)
  print(f"There are {len(inputs)} sequences.")
  return inputs, targets

In [9]:
in_, out_ = generate_training_sequences()
print(in_[0])

(19920, 128, 1)
(19920, 2)
There are 19920 sequences.
[[-1]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [-1]
 [ 0]
 [ 2]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 4]
 [-5]
 [ 1]
 [-2]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 4]
 [ 1]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 1]
 [ 0]
 [-6]
 [ 2]
 [ 1]
 [ 1]
 [ 1]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-7]
 [ 4]
 [ 1]
 [ 1]
 [ 1]
 [ 0]
 [ 1]
 [-1]
 [ 0]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [-1]
 [ 0]
 [ 1]
 [ 1]
 [ 1]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [ 2]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 1]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 1]
 [ 0]
 [ 1]
 [ 2]
 [-6]
 [ 1]
 [ 3]
 [ 0]]


In [7]:
# load the dataset, returns train and test X and y elements
def load_dataset():
  # load all train
  trainX, trainy = generate_training_sequences()
  print(trainX.shape, trainy.shape)

  # load all test
  testX, testy = generate_training_sequences()
  
  # zero-offset class values
  #trainy = trainy
  #testy = testy
  
  # one hot encode y
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, trainX, trainy

load_dataset()

(19920, 128, 1)
(19920, 2)
There are 19920 sequences.
(19920, 128, 1) (19920, 2)
(19920, 128, 1)
(19920, 2)
There are 19920 sequences.
(19920, 128, 1) (19920, 2) (19920, 128, 1) (19920, 2)


(array([[[-1],
         [ 1],
         [ 0],
         ...,
         [ 1],
         [ 3],
         [ 0]],
 
        [[ 1],
         [ 0],
         [-1],
         ...,
         [ 3],
         [ 0],
         [ 2]],
 
        [[ 0],
         [-1],
         [ 0],
         ...,
         [ 0],
         [ 2],
         [ 1]],
 
        ...,
 
        [[ 0],
         [-1],
         [ 0],
         ...,
         [-1],
         [ 0],
         [ 1]],
 
        [[-1],
         [ 0],
         [-1],
         ...,
         [ 0],
         [ 1],
         [-1]],
 
        [[ 0],
         [-1],
         [ 0],
         ...,
         [ 1],
         [-1],
         [ 0]]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=float32), array([[[-1],
         [ 1],
         [ 0],
         ...,
         [ 1],
         [ 3],
         [ 0]],
 
        [[ 1],
         [ 0],
         [-1],
         ...,
         [ 3],
         [ 0],
         [ 

In [8]:
kNumEpochs = 5
kBatchSize = 64

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, verbose=True):
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print(n_timesteps, n_features, n_outputs)

  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()

  # fit network
  model.fit(trainX, trainy, epochs=kNumEpochs, batch_size=kBatchSize, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=kBatchSize, verbose=False)
  return accuracy

# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=1):
  # load data
  trainX, trainy, testX, testy = load_dataset()#'/content/drive/MyDrive/hypertone/preprocessing')
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

# run the experiment
run_experiment()

(19920, 128, 1)
(19920, 2)
There are 19920 sequences.
(19920, 128, 1) (19920, 2)
(19920, 128, 1)
(19920, 2)
There are 19920 sequences.
(19920, 128, 1) (19920, 2) (19920, 128, 1) (19920, 2)
(19920, 128, 1) (19920, 2) (19920, 128, 1) (19920, 2)
128 1 2
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 51,102
Trainable params: 51,102
Non-trainable params: 0
_________________________________________________________________